In [ ]:
import numpy as np
import pickle

In [ ]:
x_th = -2.5
var_th = 1
min_bite_interval = 40
left_length = 48
right_length = 48

In [ ]:
def get_data(type):
    path = "C:\\ASM\\DevData\\eating\\python\\"+type+"_data.pkl"
    with open(path, "rb") as file:
        data = pickle.load(file)        
    return data    

In [ ]:
def find_min_points(x):
    step_length = min_bite_interval/2
    count = len(x)
    
    mp = []
    for i in range(0, count-step_length, step_length):
        min_index = i
        for j in range(i, i+step_length):
            if x[j] < x[min_index]:
                min_index = j
                
        if x[min_index] <= x_th and min_index>=80 and min_index<=count-80:
            mp.append(min_index)
            
    retrun mp

In [ ]:
def filter_min_points(mp, d):
    res = [];
    count = length(mp)
    for i in range(count):
        ix = mp[i]
        v = np.sum(np.var(d[ix-left_length:ix+right_length+1, 1:], axis = 0))
        if v>=var_th:
            res.append(ix)
            
    return res
    

In [ ]:
def label_min_points_by_bite(mp, annots):
    
    

In [ ]:
def label_min_points_by_meal(mp, annots):
    labels = zeros((len(mp), 1))
    if len(annots) > 0:
        
        
    mp = np.array(mp).T
    mp = mp.hstack(labels)
    return mp


In [ ]:
def find_min_points_for_dataset(ds):
    subject_count = len(d)
    ds_mp = []
    for subject in range(subject_count):
        subject_data = ds[subject]
        sess_count = len(subject_data)
        subject_mp =[]
        for sess in range(sess_count):
            sess_data = subject_data[sess]
            accel = sess_data[0]
            annots = sess_data[1]
            sess_mp = find_min_points(accel[:, 1])
            sess_mp = filter_min_points(sess_mp, accel)
            if(annots.shape[1] == 2)
                sess_mp = label_min_points_by_bite(sess_mp, annots)
            else:
                sess_mp = label_min_points_by_meal(sess_mp, annots)
                
            subject_mp.append(sess_mp)
        ds_mp.append(subject_mp)        
    return ds_mp    

In [ ]:
mps = [];
ds_names = ["uva_lab", "steven_lab", "uva_free", "steven_free"]

for i in range(4):
    ds = get_data(ds_names[i])
    mp = find_min_points_for_dataset(ds)
    mps.append(mp)

with open("C:\\ASM\\DevData\\eating\\python\\potential_min_points.pkl", "wb") as file:
    pickle.dump(mps, file)

